# Introduce Gradio from HuggingFace

In [2]:
#Import libraries
import os
import requests

from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI 

import anthropic

In [3]:
import gradio as gr

In [5]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-


In [6]:
openai = OpenAI()
system_message =  "You are a helpful, smart, kind assistant"

def message_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    completion = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
    )
    return completion.choices[0].message.content

# This can reveal the "training cut off", or the most recent date in the training data

message_gpt("What is today's date?")

"Today's date is October 10, 2023."

In [ ]:
def shout(text):
    print(f"Shout has been called with input {text}")
    return text.upper()

shout("Hello")

# gr.Interface(fn=shout, inputs="textbox", outputs="textbox").launch()

Shout has been called with input Hello
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [ ]:
# Adding share=True means that it can be accessed publically
# A more permanent hosting is available using a platform called Spaces from HuggingFace, which we will touch on next week
# NOTE: Some Anti-virus software and Corporate Firewalls might not like you using share=True. If you're at work on on a work network, I suggest skip this test.

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(share=True, inbrowser=True)

* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://2446f4fff9f5f59af6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


gio: https://2446f4fff9f5f59af6.gradio.live: Operation not supported


Shout has been called with input How can I cook a beef wellington?
